**Libraries**

In [ ]:
pip install ftfy

     |████████████████████████████████| 71kB 3.6MB/s 
  Created wheel for ftfy: filename=ftfy-5.9-cp37-none-any.whl size=46451 sha256=07f9f374fb56818ad956498cf82ca8be9e2f1573adb6aca3b5252bdd2691a7d0
  Stored in directory: /root/.cache/pip/wheels/5e/2e/f0/b07196e8c929114998f0316894a61c752b63bfa3fdd50d2fc3
Successfully built ftfy


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import re
import ftfy
import nltk
from nltk import PorterStemmer
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation, MaxPooling1D
from math import exp
from numpy import sign
from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score
from keras.models import load_model

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

**Dataset**

In [ ]:
# dfn = negative tweets
dfn=pd.read_csv('/content/drive/My Drive/FinalNegativeTweets.csv')
# dfp = positive tweets
dfp=pd.read_csv('/content/drive/My Drive/FinalPositiveTweets.csv')

In [ ]:
dfn.drop(['Unnamed: 0'],axis=1,inplace=True)
dfn.head()

,TWEETS
0,The lack of this understanding is a small but ...
1,i just told my parents about my depression and...
2,depression is something i don't speak about ev...
3,Made myself a tortilla filled with pb&j. My de...
4,@WorldofOutlaws I am gonna need depression med...


In [ ]:
dfp.drop(['Unnamed: 0'],axis=1,inplace=True)
dfp.head()

,TWEETS
0,omg its already 7:30 :O
1,Juuuuuuuuuuuuuuuuussssst Chillin!!
2,handed in my uniform today . i miss you ...
3,hmmmm.... i wonder how she my number @-)
4,thanks to all the haters up in my face all day...


In [ ]:
# size of negative tweets
dfn.shape

(2300, 1)

In [ ]:
# size of positive tweets
dfp.shape

(2300, 1)

**Preprocessing**

Removing the abbreviated words

In [ ]:
abb_List = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

abb_re = re.compile('(%s)' % '|'.join(abb_List.keys()))

def expandAbbreviations(text, abb_re=abb_re):
    def replace(match):
        return abb_List[match.group(0)]
    return abb_re.sub(replace, text)

Cleaning the tweets from special characters, digits, url's, stopwords, stemming words and punctuations

In [ ]:
def clean_tweets(tweets):
    cleaned_tweets = []
    for tweet in tweets:
        tweet = str(tweet)
        # if url links then dont append to avoid news articles
        # also check tweet length, save those > 10 (length of word "depression")
        if re.match("(\w+:\/\/\S+)", tweet) == None and len(tweet) > 10:
            #remove hashtag, @mention, emoji and image URLs
            tweet = ' '.join(re.sub("(@[A-Za-z0-9]+)|(\#[A-Za-z0-9]+)|(<Emoji:.*>)|(pic\.twitter\.com\/.*)", " ", tweet).split())
            
            #fix weirdly encoded texts
            tweet = ftfy.fix_text(tweet)
            
            #expand abbreviations
            tweet = expandAbbreviations(tweet)

            #removal of digits
            tweet = ' '.join(re.sub('\d+', '', tweet).split())

            #remove punctuation
            tweet = ' '.join(re.sub("([^0-9A-Za-z \t])", " ", tweet).split())

            #Removal of text in brackets ("text")
            tweet = " ".join(re.sub('\([^)]*\)', "",tweet).split())
        
            #Remaoval of sqaure brackets from text "[]"
            tweet = tweet.replace('[','').replace(']','')

            #specific cleaning
            tweet = tweet.replace(" 's","").replace('*','').replace(':',"").replace('&','').replace("'s","").replace("'","").replace('"','').replace(">","").replace("<","").replace("$","")

            #stop words
            stop_words = set(stopwords.words('english'))
            word_tokens = nltk.word_tokenize(tweet) 
            filtered_sentence = [w for w in word_tokens if not w in stop_words]
            tweet = ' '.join(filtered_sentence)

            #stemming words
            tweet = PorterStemmer().stem(tweet)
            
            cleaned_tweets.append(tweet)

    return cleaned_tweets

Cleaning the positive and negative tweets dataset

In [ ]:
# cnTweets = iterating the negative tweets
cnTweets = [x for x in dfn['TWEETS']]
# cpTweets = iterating the positive tweets
cpTweets= [x for x in dfp['TWEETS']]
# N_dataset = cleaned negative tweets
N_dataset = clean_tweets(cnTweets)
# P_dataset = cleaned positive tweets
P_dataset = clean_tweets(cpTweets)

In [ ]:
# splitting the long text into short
tokenizer = Tokenizer(num_words=20000)
# we take only top 20000 most frequent words
tokenizer.fit_on_texts(N_dataset+P_dataset)

In [ ]:
# storing in sequences the tokenised words
sequences_n = tokenizer.texts_to_sequences(N_dataset)
sequences_p = tokenizer.texts_to_sequences(P_dataset)

Unique words in our dataset

> Indented block



In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

Found 10762 unique tokens


In [ ]:
# bringing uniformity in words length i.e 140
data_n = pad_sequences(sequences_n, maxlen=140)
data_p = pad_sequences(sequences_p, maxlen=140)
print('Shape of negative tweets:', data_n.shape)
print('Shape of positive tweets:', data_p.shape)

Shape of negative tweets: (2261, 140)
Shape of positive tweets: (2278, 140)


Indexing word vectors

In [ ]:
# loading pretrained Word2Vec model on Google news
word2vec=KeyedVectors.load_word2vec_format("/content/drive/My Drive/GoogleNews-vectors-negative300.bin.gz", binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Preparing embedding matrix

In [ ]:
# num = no. of words
# num value we take min value btw the no. of unique words and max words if there are less unique words than the max we specified.
num = min(20000, len(word_index))+1
# dimension of embedding matrix is 300
embedding_matrix = np.zeros((num, 300))
for word, i in word_index.items():
  #.vocab = vocabulary of words
    if word in word2vec.vocab:
        embedding_matrix[i] = word2vec.word_vec(word)
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

Null word embeddings: 2418


# **Assigning** labels to positive and negative tweets i.e 1 and 0 respectively

In [ ]:
# label_N = array of assigned value 0 to the negative tweets
label_N = np.array([0] * 2300)
# label_P = array of assigned value 1 to the positive tweets
label_P = np.array([1] * 2300)

Splitting into Training 80% and Testing 20%

In [ ]:
# Negative tweets dataset data_n
perm_n = np.random.permutation(len(data_n))
# idx_train_n = negative tweets training set
idx_train_n = perm_n[:int(len(data_n)*(0.8))]
# idx_train_p = negative tweets testing set
idx_test_n = perm_n[int(len(data_n)*(0.8)):]

In [ ]:
# Positive tweets dataset data_p
perm_p = np.random.permutation(len(data_p))
# idx_train_p = positive tweets training set
idx_train_p = perm_p[:int(len(data_p)*(0.8))]
# idx_train_p = positive tweets testing set
idx_test_p = perm_p[int(len(data_p)*(0.2)):]

Combining the positive and negative tweets dataset into one dataset with their assigned labels

In [ ]:
# combined training dataset 
train_data = np.concatenate((data_n[idx_train_n], data_p[idx_train_p]))
labels_train = np.concatenate((label_N[idx_train_n], label_P[idx_train_p]))
# combined testing dataset
test_data = np.concatenate((data_n[idx_test_n], data_p[idx_test_p]))
labels_test = np.concatenate((label_N[idx_test_n], label_P[idx_test_p]))

Shuffling the positive and negative twets

In [ ]:
# training data
perm_train = np.random.permutation(len(train_data))
train_data = train_data[perm_train]
labels_train = labels_train[perm_train]
# testing data
perm_test = np.random.permutation(len(test_data))
test_data = test_data[perm_test]
labels_test = labels_test[perm_test]

Model structure 

In [ ]:
model = Sequential()
# Embedded layer
model.add(Embedding(len(embedding_matrix), 300, weights=[embedding_matrix], input_length=140, trainable=False))
# Convolutional Layer
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.2))
# LSTM Layer
model.add(LSTM(300))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

Training the model

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['acc'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 140, 300)          3228900   
_________________________________________________________________
conv1d (Conv1D)              (None, 140, 32)           28832     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 70, 32)            0         
_________________________________________________________________
dropout (Dropout)            (None, 70, 32)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 300)               399600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 3

Model summary

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3)

hist = model.fit(train_data, labels_train, epochs=10, batch_size=40, shuffle=True, callbacks=[early_stop])
model.save('/content/drive/My Drive/depression_model.h5')

Epoch 1/10
91/91 [==============================] - 28s 309ms/step - loss: 0.2387 - acc: 0.8975
Epoch 2/10
91/91 [==============================] - 28s 304ms/step - loss: 0.0607 - acc: 0.9862
Epoch 3/10
91/91 [==============================] - 28s 302ms/step - loss: 0.0434 - acc: 0.9876
Epoch 4/10
91/91 [==============================] - 28s 310ms/step - loss: 0.0368 - acc: 0.9909
Epoch 5/10
91/91 [==============================] - 28s 312ms/step - loss: 0.0302 - acc: 0.9920
Epoch 6/10
91/91 [==============================] - 29s 322ms/step - loss: 0.0208 - acc: 0.9948
Epoch 7/10
91/91 [==============================] - 28s 306ms/step - loss: 0.0160 - acc: 0.9964
Epoch 8/10
91/91 [==============================] - 28s 303ms/step - loss: 0.0121 - acc: 0.9975
Epoch 9/10
91/91 [==============================] - 28s 304ms/step - loss: 0.0133 - acc: 0.9975
Epoch 10/10
91/91 [==============================] - 28s 304ms/step - loss: 0.0181 - acc: 0.9956


Testing model

In [ ]:
labels_pred = model.predict(test_data)

In [ ]:
labels_pred.shape

(2276, 1)

In [ ]:
labels_pred = np.round(labels_pred.flatten())

In [ ]:
labels_pred.shape

(2276,)

In [ ]:
labels_pred = model.predict(test_data)
labels_pred = np.round(labels_pred.flatten())
accuracy = accuracy_score(labels_test, labels_pred)
print("Accuracy: %.2f%%" % (accuracy*100))

Accuracy: 99.38%


Result

In [ ]:
print(classification_report(labels_test, labels_pred))

              precision    recall  f1-score   support

           0       0.99      0.98      0.98       453
           1       0.99      1.00      1.00      1823

    accuracy                           0.99      2276
   macro avg       0.99      0.99      0.99      2276
weighted avg       0.99      0.99      0.99      2276



**PREDICTION ON OUR DATASET**

*Loading our model*

In [ ]:
model= load_model("/content/drive/My Drive/depression_model.h5")

#### Dataset 1 - Randywweks

In [ ]:
td1=pd.read_csv('/content/drive/My Drive/clean_randywweksF.csv')

In [ ]:
td1.head()

,after_stpw
0,go lose go humiliate embarrassed piss
1,saw facebooks
2,fire somebody need stay fire say san antonio p...
3,curious youve see pop eat shit hard today
4,new executive order mask set go effect bexar c...


In [ ]:
# Storing in List the clean tweets
T1Tweets= [x for x in td1['after_stpw']]
T_1 = clean_tweets(T1Tweets)

In [ ]:
# preprocessing
tokenizer.fit_on_texts(T_1)
sequences_T1 = tokenizer.texts_to_sequences(T_1)
word_index = tokenizer.word_index
data_T1 = pad_sequences(sequences_T1, maxlen=140)

In [ ]:
# prediction
pred1 =model.predict(data_T1)
pred1 = np.round(pred1.flatten())

In [ ]:
pred1

array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)

In [ ]:
# 0 indicates -ve sentiment and 1 indicates +ve sentiment 
(unique, counts) = np.unique(pred1, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[  0. 227.]
 [  1. 808.]]


#### Dataset 2 - NJLahori

In [ ]:
td2=pd.read_csv('/content/drive/My Drive/@NJLahoriF.csv')

In [ ]:
td2.head()

,after_stpw
0,really need spread word far wide however const...
1,thank owner lovely vase nite wishing luck trea...
2,what lovely surprise thank denzil ojosipe barc...
3,many u commit end child domestic labour many w...
4,every marriage ups challenge u overcome change...


In [ ]:
# Storing in List the clean tweets
T2Tweets= [x for x in td2['after_stpw']]
T_2 = clean_tweets(T2Tweets)

In [ ]:
# preprocessing
tokenizer.fit_on_texts(T_2)
sequences_T2 = tokenizer.texts_to_sequences(T_2)
word_index = tokenizer.word_index
data_T2 = pad_sequences(sequences_T2, maxlen=140)

In [ ]:
# prediction
pred2 =model.predict(data_T2)
pred2 = np.round(pred2.flatten())

In [ ]:
pred2

array([0., 1., 1., ..., 1., 0., 1.], dtype=float32)

In [ ]:
# 0 indicates -ve sentiment and 1 indicates +ve sentiment 
(unique, counts) = np.unique(pred2, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[  0. 161.]
 [  1. 880.]]


#### Dataset 3 - MackayEilidh

In [ ]:
td3=pd.read_csv('/content/drive/My Drive/@MackayEilidhF.csv')

In [ ]:
td3.head()

,after_stpw
0,chill room laughing joke cause im fucking hila...
1,awh thank much xx
2,awh thank much definitely symbolise much inlove
3,trouble dial not speak dial hear not alone
4,bloody love go dinner


In [ ]:
# Storing in List the clean tweets
T3Tweets= [x for x in td3['after_stpw']]
T_3 = clean_tweets(T3Tweets)

In [ ]:
# preprocessing
tokenizer.fit_on_texts(T_3)
sequences_T3 = tokenizer.texts_to_sequences(T_3)
word_index = tokenizer.word_index
data_T3 = pad_sequences(sequences_T3, maxlen=140)

In [ ]:
# prediction
pred3 =model.predict(data_T3)
pred3 = np.round(pred3.flatten())

In [ ]:
pred3

array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)

In [ ]:
# 0 indicates -ve sentiment and 1 indicates +ve sentiment 
(unique, counts) = np.unique(pred3, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[  0. 181.]
 [  1. 855.]]


#### Dataset 4 - FCancergirl

In [ ]:
td4=pd.read_csv('/content/drive/My Drive/FCancergirlF.csv')

In [ ]:
td4.head()

,after_stpw
0,day diagnose woke up completely surround love ...
1,know capacity speed up good treatment covid ti...
2,study expand medicaid not affect disparity mor...
3,enjoy content foster develop
4,since unpaid furlough not collect explore alte...


In [ ]:
# Storing in List the clean tweets
T4Tweets= [x for x in td4['after_stpw']]
T_4 = clean_tweets(T4Tweets)

In [ ]:
# preprocessing
tokenizer.fit_on_texts(T_4)
sequences_T4 = tokenizer.texts_to_sequences(T_4)
word_index = tokenizer.word_index
data_T4 = pad_sequences(sequences_T4, maxlen=140)

In [ ]:
# prediction
pred4 =model.predict(data_T4)
pred4 = np.round(pred4.flatten())

In [ ]:
pred4

array([1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.,
       1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1.,
       1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       1., 1., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0.,
       1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
       0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
       1., 0., 0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1.,
       1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0.,
       1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0.,
       1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 1., 0.,
       1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       1., 0., 1., 1., 1.

In [ ]:
# 0 indicates -ve sentiment and 1 indicates +ve sentiment 
(unique, counts) = np.unique(pred4, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[  0. 187.]
 [  1. 702.]]


#### Dataset 5 - JoeTexOnline

In [ ]:
td5=pd.read_csv('/content/drive/My Drive/JoeTexOnlineF.csv')

In [ ]:
td5.head()

,after_stpw
0,dnc virtual roll call one first example conven...
1,uhm win vote still try process thank yeah than...
2,damn caroline kennedy son look lot like jfk jr
3,raise hand want jaime harrison replace lindsey...
4,thank


In [ ]:
# Storing in List the clean tweets
T5Tweets= [x for x in td5['after_stpw']]
T_5 = clean_tweets(T5Tweets)

In [ ]:
# preprocessing
tokenizer.fit_on_texts(T_5)
sequences_T5 = tokenizer.texts_to_sequences(T_5)
word_index = tokenizer.word_index
data_T5 = pad_sequences(sequences_T5, maxlen=140)

In [ ]:
# prediction
pred5 =model.predict(data_T5)
pred5 = np.round(pred5.flatten())

In [ ]:
pred5

array([0., 1., 0., ..., 1., 1., 1.], dtype=float32)

In [ ]:
# 0 indicates -ve sentiment and 1 indicates +ve sentiment 
(unique, counts) = np.unique(pred5, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[  0. 212.]
 [  1. 804.]]


#### Dataset 6 - bigmeanjohn

In [ ]:
td6=pd.read_csv('/content/drive/My Drive/bigmeanjohnF.csv')

In [ ]:
td6.head()

,after_stpw
0,day struggle help learn what get u run chat fr...
1,comeback trail cant stop
2,finally graduate run run ha first real run yea...
3,gadsden could dial last start couldnt w
4,welcome sec


In [ ]:
# Storing in List the clean tweets
T6Tweets= [x for x in td6['after_stpw']]
T_6 = clean_tweets(T6Tweets)

In [ ]:
# preprocessing
tokenizer.fit_on_texts(T_6)
sequences_T6 = tokenizer.texts_to_sequences(T_6)
word_index = tokenizer.word_index
data_T6 = pad_sequences(sequences_T6, maxlen=140)

In [ ]:
# prediction
pred6 =model.predict(data_T6)
pred6 = np.round(pred6.flatten())

In [ ]:
pred6

array([1., 1., 1., ..., 0., 1., 0.], dtype=float32)

In [ ]:
# 0 indicates -ve sentiment and 1 indicates +ve sentiment 
(unique, counts) = np.unique(pred6, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[  0. 232.]
 [  1. 788.]]


#### Dataset 7 - bowelbabe

In [ ]:
td7=pd.read_csv('/content/drive/My Drive/bowelbabeF.csv')

In [ ]:
td7.head()

,after_stpw
0,decade cancer research lose covid
1,decade cancer research lose coronavirus patien...
2,thing worry people also not get help medical c...
3,research give hope friend whos live metastatic...
4,diagnose new guideline suggest continue exerci...


In [ ]:
# Storing in List the clean tweets
T7Tweets= [x for x in td7['after_stpw']]
T_7 = clean_tweets(T7Tweets)

In [ ]:
# preprocessing
tokenizer.fit_on_texts(T_7)
sequences_T7 = tokenizer.texts_to_sequences(T_7)
word_index = tokenizer.word_index
data_T7 = pad_sequences(sequences_T7, maxlen=140)

In [ ]:
# prediction
pred7 =model.predict(data_T7)
pred7 = np.round(pred7.flatten())

In [ ]:
pred7

array([1., 0., 1., ..., 1., 1., 0.], dtype=float32)

In [ ]:
# 0 indicates -ve sentiment and 1 indicates +ve sentiment 
(unique, counts) = np.unique(pred7, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[  0. 250.]
 [  1. 803.]]


#### Dataset 8 - christinachun

In [ ]:
td8=pd.read_csv('/content/drive/My Drive/christinachunF.csv')

In [ ]:
td8.head()

,after_stpw
0,best buddy
1,realize last airbender halloween need put blue...
2,donald trump concerned suppress vote suppress ...
3,save usps sign petition via
4,cause close heaplease sign


In [ ]:
# Storing in List the clean tweets
T8Tweets= [x for x in td8['after_stpw']]
T_8 = clean_tweets(T8Tweets)

In [ ]:
# preprocessing
tokenizer.fit_on_texts(T_8)
sequences_T8 = tokenizer.texts_to_sequences(T_8)
word_index = tokenizer.word_index
data_T8 = pad_sequences(sequences_T8, maxlen=140)

In [ ]:
# prediction
pred8 =model.predict(data_T8)
pred8 = np.round(pred8.flatten())

In [ ]:
pred8

array([1., 1., 1., ..., 0., 1., 0.], dtype=float32)

In [ ]:
# 0 indicates -ve sentiment and 1 indicates +ve sentiment 
(unique, counts) = np.unique(pred8, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[  0. 308.]
 [  1. 745.]]


#### Dataset 9 - haleycollinzzz

In [ ]:
td9=pd.read_csv('/content/drive/My Drive/haleycollinzzzF.csv')

In [ ]:
td9.head()

,after_stpw
0,averywilson video avery wilson single bilingua...
1,love penelope
2,pig
3,favorite picture funny hair use important not ...
4,love family receive mean actual world life ama...


In [ ]:
# Storing in List the clean tweets
T9Tweets= [x for x in td9['after_stpw']]
T_9 = clean_tweets(T9Tweets)

In [ ]:
# preprocessing
tokenizer.fit_on_texts(T_9)
sequences_T9 = tokenizer.texts_to_sequences(T_9)
word_index = tokenizer.word_index
data_T9 = pad_sequences(sequences_T9, maxlen=140)

In [ ]:
# prediction
pred9 =model.predict(data_T9)
pred9 = np.round(pred9.flatten())

In [ ]:
pred9

array([1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 0.,
       1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1.,
       1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1.,
       0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0.,
       0., 1., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0.,
       1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 0.,
       1., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1.,
       1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [ ]:
# 0 indicates -ve sentiment and 1 indicates +ve sentiment 
(unique, counts) = np.unique(pred9, return_counts=True)
frequencies = np.asarray((unique, counts)).T

print(frequencies)

[[  0. 273.]
 [  1. 696.]]


**Dataset 10: NONotTHatKENNYG**

In [ ]:
td10=pd.read_csv('/content/drive/My Drive/NoNotThatKennyGF.csv')

In [ ]:
td10.head()

,after_stpw
0,year ago spend first night hospital acute care...
1,week immunotherapy start far bad side effect f...
2,brother law brazilian chair national latino st...
3,encourage blood work yesterday cancer antigen ...
4,begin


In [ ]:
# Storing in List the clean tweets
T10Tweets= [x for x in td10['after_stpw']]
T_10 = clean_tweets(T10Tweets)

In [ ]:
# preprocessing
tokenizer.fit_on_texts(T_10)
sequences_T10 = tokenizer.texts_to_sequences(T_10)
word_index = tokenizer.word_index
data_T10 = pad_sequences(sequences_T10, maxlen=140)

In [ ]:
# prediction
pred10 =model.predict(data_T10)
pred10 = np.round(pred10.flatten())

In [ ]:
pred10

array([1., 0., 1., ..., 0., 1., 0.], dtype=float32)

Dataset:clean-p1F.csv
---



In [ ]:
td11=pd.read_csv('/content/drive/My Drive/clean-p1F.csv')

In [ ]:

td11.head()

,after_stpw
0,accord al gore water stop b earth always change
1,climate change hoax
2,ask post source deflection dolt
3,bomg what lie pos democrat scream bs no eviden...
4,post claim bs trump sue defamation win


In [ ]:
# Storing in List the clean tweets
T11Tweets= [x for x in td11['after_stpw']]
T_11 = clean_tweets(T11Tweets)

In [ ]:
# preprocessing
tokenizer.fit_on_texts(T_11)
sequences_T11 = tokenizer.texts_to_sequences(T_11)
word_index = tokenizer.word_index
data_T11 = pad_sequences(sequences_T11, maxlen=140)

In [ ]:
# prediction
pred11 =model.predict(data_T11)
pred11 = np.round(pred11.flatten())

In [ ]:
pred11

array([1., 1., 1., ..., 1., 1., 1.], dtype=float32)

Dataset: clean-p3F.csv

In [ ]:
td12=pd.read_csv('/content/drive/My Drive/clean-p3F.csv')

In [ ]:
td12

,after_stpw
0,maybe look excuse replace tire bent rim
1,depends america yea
2,ever get feel google know much
3,drive around toronto cab give absolute fuck ba...
4,project recently fill newcrest sims custom hom...
...,...
391,balance soon enough stroong drug might bit man...
392,clock high rate transition go take downtown
393,oh wonderful news congrats kate
394,dunno viewpoint necessarily change much think ...


In [ ]:
# Storing in List the clean tweets
T12Tweets= [x for x in td12['after_stpw']]
T_12 = clean_tweets(T12Tweets)

In [ ]:
# preprocessing
tokenizer.fit_on_texts(T_12)
sequences_T12 = tokenizer.texts_to_sequences(T_12)
word_index = tokenizer.word_index
data_T12 = pad_sequences(sequences_T12, maxlen=140)

In [ ]:
# prediction
pred12 =model.predict(data_T12)
pred12 = np.round(pred12.flatten())

In [ ]:
pred12

array([1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1.,
       1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0.,
       1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1.,
       0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0.,
       1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0.,
       0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0.,
       0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1.,
       1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1.,
       0., 1., 0., 1., 0.

**Dataset: clean-p4F.csv**

In [ ]:
td13=pd.read_csv('/content/drive/My Drive/clean-p4F.csv')

In [ ]:
td13

,after_stpw
0,go pleasee
1,one nurse french braid hair icu felt love take...
2,amazing friend casey never see star war behold
3,make eliza schuyler disney princess
4,amber hear literally admits abuse johnny depp ...
...,...
1787,alexfromtarget ellen
1788,life goal
1789,terry not finish product get well want keep ge...
1790,mind go real quick


In [ ]:
# Storing in List the clean tweets
T13Tweets= [x for x in td13['after_stpw']]
T_13 = clean_tweets(T13Tweets)

In [ ]:
# preprocessing
tokenizer.fit_on_texts(T_13)
sequences_T13 = tokenizer.texts_to_sequences(T_13)
word_index = tokenizer.word_index
data_T13 = pad_sequences(sequences_T13, maxlen=140)

In [ ]:
# prediction
pred13 =model.predict(data_T13)
pred13 = np.round(pred13.flatten())

In [ ]:
pred13

array([0., 1., 0., ..., 0., 1., 1.], dtype=float32)

**Dataset: 14**

In [ ]:
td14=pd.read_csv('/content/drive/My Drive/clean-p5F.csv')

In [ ]:
td14.head()

,after_stpw
0,get know womeninscience james join u thank won...
1,agree inspire leadership approach love watch c...
2,love idea
3,miracle let u know help
4,indeed thank


In [ ]:
# Storing in List the clean tweets
T14Tweets= [x for x in td14['after_stpw']]
T_14 = clean_tweets(T14Tweets)

In [ ]:
# preprocessing
tokenizer.fit_on_texts(T_14)
sequences_T14 = tokenizer.texts_to_sequences(T_14)
word_index = tokenizer.word_index
data_T14 = pad_sequences(sequences_T14, maxlen=140)

In [ ]:
# prediction
pred14 =model.predict(data_T14)
pred14 = np.round(pred14.flatten())

In [ ]:
pred14

array([0., 1., 1., ..., 1., 1., 1.], dtype=float32)

**Dataset 15**

In [ ]:
td15=pd.read_csv('/content/drive/My Drive/clean-p6F.csv')

In [ ]:
td15.head()

,after_stpw
0,first timers new comer take win best sirloin s...
1,filthy filly open restaurant offering bottomle...
2,someone quick government shut get stanew one i...
3,filthy filly crankin cranberry salsa top lemon...
4,old town family christmas market place parade ...


In [ ]:
# Storing in List the clean tweets
T15Tweets= [x for x in td15['after_stpw']]
T_15 = clean_tweets(T15Tweets)

In [ ]:
# preprocessing
tokenizer.fit_on_texts(T_15)
sequences_T15 = tokenizer.texts_to_sequences(T_15)
word_index = tokenizer.word_index
data_T15 = pad_sequences(sequences_T15, maxlen=140)

In [ ]:
# prediction
pred15 =model.predict(data_T15)
pred15 = np.round(pred15.flatten())

In [ ]:
pred15

array([1., 1., 0., ..., 1., 1., 1.], dtype=float32)

**Dataset: clean-p7F.csv**

In [ ]:
td16=pd.read_csv('/content/drive/My Drive/clean-p7F.csv')

In [ ]:
td16

,after_stpw
0,brotherly love mycity
1,make take phila tough allove
2,either die hero live long enough see become vi...
3,whats go whats happen
4,man dont even know start started play ball kob...
...,...
2534,good ya
2535,sleep
2536,follow
2537,sleep tired


In [ ]:
# Storing in List the clean tweets
T16Tweets= [x for x in td16['after_stpw']]
T_16 = clean_tweets(T16Tweets)

In [ ]:
# preprocessing
tokenizer.fit_on_texts(T_16)
sequences_T16 = tokenizer.texts_to_sequences(T_16)
word_index = tokenizer.word_index
data_T16 = pad_sequences(sequences_T16, maxlen=140)

In [ ]:
# prediction
pred16 =model.predict(data_T16)
pred16 = np.round(pred16.flatten())

In [ ]:

pred16

array([0., 1., 0., ..., 1., 1., 1.], dtype=float32)

**Dataset: clean-p8F.csv**

In [ ]:
td17=pd.read_csv('/content/drive/My Drive/clean-p8F.csv')

In [ ]:
td17.head()

,after_stpw
0,ask deliver year brain cancer survivor shirt m...
1,today year anniversary brain cancer survivor huge
2,today year anniversary cancer survivor huge
3,merry christmas happy holiday
4,really enjoy youocomnts


In [ ]:
# Storing in List the clean tweets
T17Tweets= [x for x in td17['after_stpw']]
T_17= clean_tweets(T17Tweets)

In [ ]:
# preprocessing
tokenizer.fit_on_texts(T_17)
sequences_T17 = tokenizer.texts_to_sequences(T_17)
word_index = tokenizer.word_index
data_T17 = pad_sequences(sequences_T17, maxlen=140)

In [ ]:
# prediction
pred17 =model.predict(data_T17)
pred17 = np.round(pred17.flatten())

In [ ]:
pred17

array([1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
       1., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1.],
      dtype=float32)

**Dataset: clean-p9F.csv**

In [ ]:
td18=pd.read_csv('/content/drive/My Drive/clean-p9F.csv')

In [ ]:
td18.head()

,after_stpw
0,cc
1,every remember go organizer digital strategist...
2,not take u trumpbudget would cut medicare medi...
3,lie
4,lie


In [ ]:
# Storing in List the clean tweets
T18Tweets= [x for x in td18['after_stpw']]
T_18= clean_tweets(T18Tweets)

In [ ]:
# preprocessing
tokenizer.fit_on_texts(T_18)
sequences_T18 = tokenizer.texts_to_sequences(T_18)
word_index = tokenizer.word_index
data_T18 = pad_sequences(sequences_T18, maxlen=140)

In [ ]:
# prediction
pred18 =model.predict(data_T18)
pred18 = np.round(pred18.flatten())

In [ ]:
pred18

array([1., 1., 1., ..., 0., 1., 0.], dtype=float32)

**Dataset: clean-p10F.csv**

In [ ]:
td19=pd.read_csv('/content/drive/My Drive/clean-p10F.csv')

In [ ]:
td19.head()
td19.shape

(3679, 1)

In [ ]:
# Storing in List the clean tweets
T19Tweets= [x for x in td19['after_stpw']]
T_19= clean_tweets(T19Tweets)


In [ ]:
# preprocessing
tokenizer.fit_on_texts(T_19)
sequences_T19 = tokenizer.texts_to_sequences(T_19)
word_index = tokenizer.word_index
data_T19 = pad_sequences(sequences_T19, maxlen=140)

In [ ]:
# prediction
pred19 =model.predict(data_T19)
pred19 = np.round(pred19.flatten())

In [ ]:
pred19.shape

(2733,)

In [ ]:
output1=pd.DataFrame()
output1["tweet"]=T_19
output1["output"]=pred19
#enter the path of the output file with name
output1.to_csv('/content/drive/My Drive/Results19.csv', index = False)

In [ ]:
output1=pd.DataFrame()
output1["tweet"]=T_18
output1["output"]=pred18
#enter the path of the output file with name
output1.to_csv('/content/drive/My Drive/Results18.csv', index = False)